In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [79]:
# wrong order + right relation = 0.16666666666666669
s1 = np.array([
    [[20, 10, 0], [50, 30, 100]],
    [[50, 30, 100], [180, 20, 20]],
    [[180, 20, 20], [30, 15, 0]]
], dtype=np.double)

s2 = np.array([
    [[180, 20, 20], [20, 10, 5]],
    [[20, 10, 5], [30, 16, 1]],
    [[30, 16, 1], [50, 29, 120]]
], dtype=np.double)

In [69]:
# right order + right relation = 0.0
s1 = np.array([
    [[20, 10, 0], [50, 30, 100]],
    [[50, 30, 100], [180, 20, 20]],
    [[180, 20, 20], [30, 15, 0]]
], dtype=np.double)

s2 = np.array([
    [[18, 12, 5], [47, 35, 110]],
    [[47, 35, 110], [200, 18, 40]],
    [[200, 18, 40], [30, 9, 4]]
], dtype=np.double)

In [60]:
# right order + wrong relation = 0.16666666666666663
s1 = np.array([
    [[20, 10, 0], [50, 30, 100]],
    [[50, 30, 100], [180, 20, 20]],
    [[180, 20, 20], [30, 15, 0]]
], dtype=np.double)

s2 = np.array([
    [[20, 10, 20], [180, 20, 100]],
    [[180, 20, 100], [50, 35, 40]],
    [[50, 35, 40], [30, 15, 30]]
], dtype=np.double)

In [81]:
# tf.reset_default_graph()
    
# 获取数据形状
no, seq_len, dim = np.asarray(s1).shape    
    
# 提取时间步数
ori_time = np.array([seq_len] * no)
generated_time = np.array([seq_len] * no)
max_seq_len = seq_len
    
# 网络参数
hidden_dim = int(dim / 2)

In [82]:
rnn_name = 'gru'
model = tf.keras.Sequential()
rnn_layer = tf.keras.layers.GRU(hidden_dim, activation='tanh', name='d_cell')

model.add(rnn_layer)
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy)

In [83]:
# 标签：1 表示真实数据，0 表示生成数据
y_real = np.ones((no, 1))
y_fake = np.zeros((no, 1))

In [89]:
 # 合并数据
X_train = np.concatenate([s1, s2], axis=0)
# print(X_train.shape)
# print(X_train)

y_train = np.concatenate([y_real, y_fake], axis=0)
# print(y_train.shape)
# print(y_train)

In [85]:
 # 训练判别器
iterations = 10
model.fit(X_train, y_train, epochs=iterations, batch_size=no, verbose=0)
    

In [86]:
# 测试集上计算结果
y_pred_real = model.predict(s1)
print(y_pred_real)
y_pred_fake = model.predict(s2)
print(y_pred_fake)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
[[0.22656076]
 [0.22658843]
 [0.7583976 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[[0.73299974]
 [0.7735896 ]
 [0.22685586]]


In [87]:
# 拼接真实和伪造数据的预测结果
y_pred_final = np.squeeze(np.concatenate((y_pred_real, y_pred_fake), axis=0))
y_label_final = np.concatenate((np.ones([len(y_pred_real),]), np.zeros([len(y_pred_fake),])), axis=0)


In [88]:
# 计算准确率 (lower better)
acc = accuracy_score(y_label_final, (y_pred_final > 0.5))
discriminative_score = np.abs(0.5 - acc)
print(discriminative_score)

0.16666666666666669
